In [ ]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageTk

# Load the saved model
model = load_model('final_model_weights.hdf5')  # Replace with the actual path to your model file

# Define the emotions
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

root = tk.Tk()
root.title('Emotion Detection')
root.geometry('800x600')
root.configure(background='black')
root.resizable(False, False)

# Add 'Emotion Detection' label at the top of the window
title_label = tk.Label(root, text='Emotion Detection', bg='black', fg='white', font=('arial', 18, 'bold'))
title_label.pack(pady=20)

# Create a function to open and process the image
def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Load a pre-trained face detection model
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            resized_face = cv2.resize(face, (48, 48))
            normalized_face = resized_face / 255.0
            reshaped_face = np.reshape(normalized_face, (1, 48, 48, 1))
            
            # Predict emotion using the loaded model
            prediction = model.predict(reshaped_face)
            emotion_index = np.argmax(prediction)
            predicted_emotion = emotions[emotion_index]
            
            cv2.putText(image, predicted_emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image.thumbnail((400, 400))  # Resize the image for display
        img = ImageTk.PhotoImage(image=image)
        
        image_label.config(image=img)
        image_label.image = img
        
        predicted_label.config(text=f"Predicted Emotion: {predicted_emotion}")

        # Display the predicted image next to the loaded image
        predicted_image = cv2.resize(image, (400, 400))
        predicted_image = Image.fromarray(predicted_image)
        predicted_image = ImageTk.PhotoImage(image=predicted_image)
        predicted_label.config(image=predicted_image)
        predicted_label.image = predicted_image

# Create a label to display the loaded image
image_label = tk.Label(root, bg='black')
image_label.place(x=50, y=100)

# Create a label to display the predicted image
predicted_label = tk.Label(root, bg='black')
predicted_label.place(x=450, y=100)

# Create a button to open the image
open_button = tk.Button(root, text='Open Image', command=open_image, bg='white', fg='black', font=('arial', 12, 'bold'))
open_button.place(x=350, y=500)

root.mainloop()
